# ... what on such booring day to do?

In [1]:
! echo "Ähmm, what a day?"; date

Ähmm, what a day?
Sun Jul 26 18:12:05 UTC 2020


In [23]:
! echo 'start in progress   ¯\_(ツ)_/¯  (3 sec to kill by ctrl-c)'
! sleep 3

start in progress   ¯\_(ツ)_/¯  (3 sec to kill by ctrl-c)


ok!, & what to do? - Looking for freebeds in state of thuringia!


so we need some hammer and steel .. hammer:

In [2]:
! echo "hammer? -ah, a hammer:";  
! curl -s https://webinstall.dev/jq | bash

hammer? -ah, a hammer:

Thanks for using webi to install 'jq' on 'linux/amd64'.
Have a problem? Experience a bug? Please let us know:
        https://github.com/webinstall/packages/issues

/home/jovyan/Downlo 100%[===================>]   3.77M  16.0MB/s    in 0.2s    
Moving /home/jovyan/Downloads/jq-linux64
Installing to /home/jovyan/.local/opt/jq-v1.6/bin/jq}
/home/jovyan/Downlo     [  <=>               ] 640.39K  2.82MB/s    in 0.2s    
Installed 'jq' v1.6 as /home/jovyan/.local/bin/jq



In [3]:
! echo "Is the hammer ready now?"; jq --help

Is the hammer ready now?
jq - commandline JSON processor [version 1.6]

Usage:	jq [options] <jq filter> [file...]
	jq [options] --args <jq filter> [strings...]
	jq [options] --jsonargs <jq filter> [JSON_TEXTS...]

jq is a tool for processing JSON inputs, applying the given filter to
its JSON text inputs and producing the filter's results as JSON on
standard output.

The simplest filter is ., which copies jq's input to its output
unmodified (except for formatting, but note that IEEE754 is used
for number representation internally, with all that that implies).

For more advanced filters see the jq(1) manpage ("man jq")
and/or https://stedolan.github.io/jq

Example:

	$ echo '{"foo": 0}' | jq .
	{
		"foo": 0
	}

Some of the options include:
  -c               compact instead of pretty-printed output;
  -n               use `null` as the single input value;
  -e               set the exit status code based on the output;
  -s               read (slurp) all inputs into an array; apply filte

this lightwight hammer seems as a good working tool, ok.

to hammer on it, we need also steel.

"Morgenpost" say's had some steel for us:

In [4]:
! curl curl https://interaktiv.morgenpost.de/corona-deutschland-intensiv-betten-monitor-krankenhaus-auslastung/data/intensivregister.json | tail

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: curl
100   365    0   365    0     0    498      0 --:--:-- --:--:-- --:--:-- 26071
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://interaktiv.morgenpost.de/data/corona/intensivregister.json">here</a>.</p>
<hr>
<address>Apache/2.2.22 (Debian) Server at interaktiv.morgenpost.de Port 443</address>
</body></html>


In [5]:
! echo "So go fw 2 this:"; curl https://interaktiv.morgenpost.de/data/corona/intensivregister.json | tail

So go fw 2 this:
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1165k  100 1165k    0     0   837k      0  0:00:01  0:00:01 --:--:--  836k
				null
			],
			"behandlungsschwerpunktL3": [
				null
			],
			"label": "Hufeland Klinikum, Mühlhausen",
			"faelleCovidAktuell": 2
		}
	]
}

we are lucky!

In [6]:
! curl https://interaktiv.morgenpost.de/data/corona/intensivregister.json -o BettenAllDE.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1165k  100 1165k    0     0   847k      0  0:00:01  0:00:01 --:--:--  848k


 
 Problem   Problem  Problem   Problem  Problem   Problem  
 Problem   Problem  Problem   Problem  Problem   Problem  

In [20]:
! cat BettenAllDE.json | jq '.data[] reduce .[] .faelleCovidAktuell as $item (0; . + $item)'

jq: error: syntax error, unexpected reduce, expecting $end (Unix shell quoting issues?) at <top-level>, line 1:
.data[] reduce .[] .faelleCovidAktuell as $item (0; . + $item)        
jq: 1 compile error
cat: write error: Broken pipe


what the heck?

in jq  https://jqplay.org/# play ok
but not in cli ...:

reduce .label[] .foo as $item (0; . + $item)

***

in:

{"label":
[
{"foo": 10},
{"foo": 21},
{"foo": 42}
]}


out:

73

***

in:

! cat BettenAllDE.json | jq '.data[] reduce .[] .faelleCovidAktuell as $item (0; . + $item)'

out:

... unexpected reduce, expecting $end ... (-> above)

***

so we must go by shell awk ...
not at all
but would be nice somebody find out a solution only with jq!

In [14]:
! cat BettenAllDE.json | jq '.data[] | select(.krankenhausStandort.bundesland == "THUERINGEN") | ( .faelleCovidAktuell | tostring ) '

"3"
"0"
"3"
"2"
"3"
"null"
"8"
"1"
"null"
"0"
"null"
"null"
"null"
"0"
"0"
"4"
"0"
"1"
"0"
"7"
"2"
"2"
"1"
"5"
"0"
"0"
"null"
"0"
"null"
"null"
"5"
"3"
"0"
"null"
"0"
"0"
"2"


In [7]:
! cat BettenAllDE.json | jq '.data[] | select(.krankenhausStandort.bundesland == "THUERINGEN") | .krankenhausStandort.plz + " " + .meldezeitpunkt + " co= " + ( .faelleCovidAktuell | tostring ) + " " + .label + "   HighCare: " + .bettenStatus.statusHighCare' > BettenThur.txt

In [15]:
! cat BettenAllDE.json | jq '.data[] | select(.krankenhausStandort.bundesland == "THUERINGEN") | .faelleCovidAktuell  '

3
0
3
2
3
null
8
1
null
0
null
null
null
0
0
4
0
1
0
7
2
2
1
5
0
0
null
0
null
null
5
3
0
null
0
0
2


In [19]:
! ! cat BettenAllDE.json | jq '.data[] | select(.krankenhausStandort.bundesland == "THUERINGEN") | .faelleCovidAktuell  ' | awk '{s+=$1} END {print s}' 

52


the mystery number to find out fake ... ;) (he, why are this 10 to much?)

now save the number somewhere ..., some poc, not realy good

In [25]:
!   hostname > alive.txt ;

!    memf=$(awk -F"[: ]+" '/MemFree/ {print $2;exit}' /proc/meminfo); 

!    echo $memf >> alive.txt ;

!   uptime >> alive.txt ; 

!   date >> alive.txt ; 

!    echo "alive after init" >> alive.txt ; 

!    cat alive.txt | curl -s -F 'f:1=<-' jor2:pjor2@ix.io  | tee -a alive.txt;

!    # lock at http://ix.io/user/jor2 for result


http://ix.io/2snj


In [26]:
! cat BettenAllDE.json | jq '.data[] | select(.krankenhausStandort.bundesland == "THUERINGEN") | .krankenhausStandort.plz + " " + .meldezeitpunkt + " co= " + ( .faelleCovidAktuell | tostring ) + " " + .label + "   HighCare: " + .bettenStatus.statusHighCare' > BettenThur.txt

is some steel in the "Morgenpost" in the morning?

In [8]:
! tail BettenThur.txt

"99706 2020-07-26T05:07:19Z co= 0 KMG Klinikum Sondershausen   HighCare: "
"99734 2020-07-26T08:31:03Z co= null Südharz Klinikum Nordhausen gGmbH   HighCare: VERFUEGBAR"
"99752 2020-07-26T05:52:29Z co= null Helios Klinik Bleicherode, Bleicherode   HighCare: "
"99768 2020-07-26T06:48:05Z co= 5 Lungenklinik Neustadt   HighCare: VERFUEGBAR"
"99817 2020-07-26T07:36:45Z co= 3 St. Georg Klinikum Eisenach   HighCare: VERFUEGBAR"
"99867 2020-07-26T04:19:48Z co= 0 Helios Klinikum Gotha   HighCare: "
"99894 2020-07-26T07:29:28Z co= null SRH Krankenhaus Waltershausen-Friedrichroda GmbH, Standort Friedrichroda   HighCare: VERFUEGBAR"
"99947 2020-07-26T05:15:41Z co= 0 Hufeland Klinikum, Bad Langensalza   HighCare: VERFUEGBAR"
"99955 2020-07-26T06:46:50Z co= 0 MEDIAN Klinik Bad Tennstedt, Hauptstandort   HighCare: "
"99974 2020-07-26T05:17:14Z co= 2 Hufeland Klinikum, Mühlhausen   HighCare: VERFUEGBAR"


***

Dear people in Neustadt, Eisenach and Mühlhausen: 

### We wich you best healthy!

***

In [9]:
! touch BITHLi && sort BettenThur.txt | nl | tee -a BITHLi | wc -l >> BITHLi && echo `date` >> BITHLi ; cat BITHLi

     1	"04600 2020-07-26T07:08:28Z co= 3 Klinikum Altenburger Land   HighCare: VERFUEGBAR"
     2	"07318 2020-07-26T08:51:09Z co= 0 Georgius Agricola   HighCare: VERFUEGBAR"
     3	"07548 2020-07-26T05:49:33Z co= 3 SRH Wald-Klinikum Gera   HighCare: VERFUEGBAR"
     4	"07607 2020-07-26T05:21:45Z co= 2 Waldkliniken Eisenberg   HighCare: VERFUEGBAR"
     5	"07747 2020-07-26T03:17:15Z co= 3 Universitätsklinikum Jena Lobeda   HighCare: VERFUEGBAR"
     6	"07907 2020-07-26T05:57:38Z co= null Kreiskrankenhaus Schleiz GmbH   HighCare: VERFUEGBAR"
     7	"07973 2020-07-26T05:54:52Z co= 8 Kreiskrankenhaus Greiz   HighCare: VERFUEGBAR"
     8	"36433 2020-07-26T05:53:09Z co= 1 Klinikum Bad Salzungen   HighCare: VERFUEGBAR"
     9	"36448 2020-07-24T06:21:42Z co= 0 m&i-Fachklinik Bad Liebenstein   HighCare: NICHT_VERFUEGBAR"
    10	"36448 2020-07-26T06:43:45Z co= null MEDIAN Heinrich-Mann-Klinik, MEDIAN Heinrich-Mann-Klinik Bad Liebenstein   HighCare: "
    11	"37308 2020-07-26T08:03:24Z co= null E

we will not count the people WITH covid, 

only we SHOW as example working with the tool

Our best wishes to the people in: 

Altenburg, Gera, Jena, Eisenberg, Greiz, Bad Salzungen, Suhl, Meiningen, Arnstadt, Bad Berka, Weimar, Neustadt, Eisenach und Mühlhausen! 

## Be quick healthy again!

Every suggestion, mostly for code, are very welcome. 

## Keep healthy!

In [10]:
! date ; echo "this was all for the 1st ... CU! :)  "

Sun Jul 26 18:12:20 UTC 2020
this was all for the 1st ... CU! :)  
